In [1]:
# 구현하려는 것
# https://github.com/vinhkhuc/MemN2N-babi-python
# https://github.com/nmhkahn/MemN2N-pytorch

In [2]:
from __future__ import print_function

In [3]:
import mxnet as mx
ctx = mx.cpu()

In [4]:
import os
import glob
data_dir = glob.glob('/home/hyungjunkim/Dropbox/repo/dl/MemN2N/scripts/' + '/data/')[0]

!cd /home/hyungjunkim/Dropbox/repo/dl/MemN2N/scripts/data && \  
wget https://s3.amazonaws.com/text-datasets/babi_tasks_1-20_v1-2.tar.gz && \  
tar xvf babi_tasks_1-20_v1-2.tar.gz

In [5]:
from datautils import *

In [6]:
task_id = 1

In [7]:
train_data = bAbIDataset(data_dir + 'tasks_1-20_v1-2/en/', task_id=task_id, train=True)

/usr/lib/python3.5/re.py:203: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [8]:
test_data = bAbIDataset(data_dir + 'tasks_1-20_v1-2/en/', task_id=task_id, train=False)

/usr/lib/python3.5/re.py:203: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [9]:
word_re = {value:key for key, value in train_data.word_idx.items()}

In [10]:
word_re 

{1: 'back',
 10: 'kitchen',
 11: 'mary',
 12: 'moved',
 13: 'office',
 14: 'sandra',
 15: 'the',
 16: 'to',
 17: 'travelled',
 18: 'went',
 19: 'where',
 2: 'bathroom',
 3: 'bedroom',
 4: 'daniel',
 5: 'garden',
 6: 'hallway',
 7: 'is',
 8: 'john',
 9: 'journeyed',
 'time1': 'time1',
 'time10': 'time10',
 'time2': 'time2',
 'time3': 'time3',
 'time4': 'time4',
 'time5': 'time5',
 'time6': 'time6',
 'time7': 'time7',
 'time8': 'time8',
 'time9': 'time9'}

In [11]:
for ii, (x,y,z) in enumerate(train_data):
    for idx in x:
        print (idx)
        print (" ".join([word_re.get(int(i), '.') for i in idx if int(i) != 0]))
    
    print (" ".join([word_re.get(int(i), '?') for i in y if int(i) != 0]))
    print (word_re.get(int(z), '?'))
    
    print ('\n')
    if ii == 2:
        break

[11 12 16 15  2  0 21]
mary moved to the bathroom .
[ 8 18 16 15  6  0 20]
john went to the hallway .
[0 0 0 0 0 0 0]

[0 0 0 0 0 0 0]

[0 0 0 0 0 0 0]

[0 0 0 0 0 0 0]

[0 0 0 0 0 0 0]

[0 0 0 0 0 0 0]

[0 0 0 0 0 0 0]

[0 0 0 0 0 0 0]

where is mary
bathroom


[11 12 16 15  2  0 23]
mary moved to the bathroom .
[ 8 18 16 15  6  0 22]
john went to the hallway .
[ 4 18  1 16 15  6 21]
daniel went back to the hallway .
[14 12 16 15  5  0 20]
sandra moved to the garden .
[0 0 0 0 0 0 0]

[0 0 0 0 0 0 0]

[0 0 0 0 0 0 0]

[0 0 0 0 0 0 0]

[0 0 0 0 0 0 0]

[0 0 0 0 0 0 0]

where is daniel
hallway


[11 12 16 15  2  0 25]
mary moved to the bathroom .
[ 8 18 16 15  6  0 24]
john went to the hallway .
[ 4 18  1 16 15  6 23]
daniel went back to the hallway .
[14 12 16 15  5  0 22]
sandra moved to the garden .
[ 8 12 16 15 13  0 21]
john moved to the office .
[14  9 16 15  2  0 20]
sandra journeyed to the bathroom .
[0 0 0 0 0 0 0]

[0 0 0 0 0 0 0]

[0 0 0 0 0 0 0]

[0 0 0 0 0 0 0]

where is da

In [12]:
inputs_train, queries_train, answers_train = train_data.data_story, train_data.data_query, train_data.data_answer
inputs_test, queries_test, answers_test = test_data.data_story, test_data.data_query, test_data.data_answer

In [13]:
def position_encoding(sentences, emb_dim):
    s_tmp = sentences.asnumpy()
    n_row,n_col = np.shape(s_tmp)
    for i in range(n_row):
        for j in range(n_col):
            s_tmp[i,j] = (1-j/n_col) - (n_row/emb_dim)*(1-2*j/n_col)    
    return mx.nd.array(s_tmp, ctx).expand_dims(2)

## End-To-End Memory Network (N2NMN)

i = 문장   
j = 단어

Inputs: $x_{i}$: 문장(context)  
$q$: 질의  
$target$: 답변(a word or words)

![img0](imgs/n2nmn_fig0.png)

#### 메모리 벡터: 임베딩된 워드의 합을 문장으로 표기

$m_{i}={\sum}_{j}Ax_{i,j}$

#### Attention $P_{i}$: 질의(q)와 가장 관련된 Senence(context) 확률로 나타냄

$P_i=Softmax(uT {\cdot} mi)$

$o={\sum}_{i}p_{i}c_{i}$

$a=Softmax(W(o+u))$

internal state $u={\sum}_{j}Bq_{j}$

#### 학습: A, C, B, W

#### multi-layer

adjacent: output embedding(o1 + u1)이 새로운 input embedding이 됨   
Layer-wise (RNN-like): 모든 레이어가 A와 C를 공유함.  $uk^{+1} = Hu^{k}+o^{k}$, linear mapping H

#### 성능 올리기: 1. Senetence Representation, 2. Terporal Encoding, 3. Lerning time invariance by injection random noise

In [14]:
print (np.shape(inputs_train))
print (np.shape(queries_train))
print (np.shape(answers_train))

(1000, 10, 7)
(1000, 7)
(1000,)


In [15]:
import mxnet as mx
from mxnet import autograd, gluon, nd, init
from mxnet.gluon import Block, nn


class MemN2N(Block):
    def __init__(self, config, **kwargs):
        super(MemN2N, self).__init__(**kwargs)

        ## 고정
        self.vocab_size = len(word_re)
        
        ## 
        self.init_std = 0.015
        self.nhop = 3
        self.emb_dim = 50

        with self.name_scope():
            
            # Embedding A (메모리 벡터)
            self.A = nn.Embedding(input_dim=self.vocab_size, 
                                  output_dim=self.emb_dim, 
                                  weight_initializer=init.Normal(self.init_std))
             
            # Embedding C (컨텍스트 벡터)
            self.C = nn.Embedding(input_dim=self.vocab_size,
                                  output_dim=self.emb_dim, 
                                  weight_initializer=init.Normal(self.init_std))
            
            # Embedding Q (쿼리 벡터)
            self.B = nn.Embedding(input_dim=self.vocab_size, 
                                  output_dim=self.emb_dim, 
                                  weight_initializer=init.Normal(self.init_std))

            # Final Predict 
            self.W = nn.Dense(self.vocab_size, weight_initializer=init.Normal(self.init_std))
            
    def forward(self, sentences, question):

        hid = []
        hid.append(question)
        
        # 16, 50, 7 , 19 > 16, 50, 19
        m_i = self.A(sentences) #* position_encoding(sentences, self.emb_dim)
        m_i = m_i.sum(axis=2)
        
        # 16, 50, 7 , 19 > 16, 50, 19
        c_i = self.C(sentences)
        c_i = c_i.sum(axis=2)
            
        for n in range(self.nhop):
            
            if n == 0:
                #  16, 7, 50 > 16, 50
                u = self.B(hid[-1])# * position_encoding(hid[-1], self.emb_dim)
                u = u.sum(axis=1)
            else:
                u = hid[-1]

            # [16, 1, 50] [16, 50, 10]  > [16, 1, 50]
            m_out = nd.batch_dot(u.expand_dims(1), m_i.swapaxes(1,2))
            P = nd.softmax(m_out, axis=2)
            
            #Cout = P * c_i
            # [16, 1, 10] [16, 10, 50] > [16, 1, 50]
            Cout = nd.batch_dot(P,  c_i)
            # [16, 1, 50] > [16, 50]
            o = Cout.reshape((0,-1))#Cout.sum(axis=(1))
            
            ## O + U [16, 50]
            Dout = o + u
            hid.append(Dout)
           
        z = self.W(hid[-1])
        return z

In [16]:
def train(model, trainer, x_input, x_query, x_answers, batch_size):
    softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()
    max_grad_norm=40
    N = int(math.ceil(len(x_input) / batch_size))
    cost = 0.0

    for idx in range(N):
        with autograd.record():
            out = model(x_input, x_query)
            loss = softmax_cross_entropy(out, x_answers)
            loss.backward()
            
        grads = [i.grad() for i in model.collect_params().values()]
        gluon.utils.clip_global_norm(grads, max_grad_norm)
        trainer.step(batch_size)
        cost += nd.sum(loss).asscalar()

    return cost/N/batch_size

In [17]:
model = MemN2N(None)
model.collect_params().initialize(mx.init.Xavier(),ctx=ctx)

In [18]:
print (model)

MemN2N(
  (C): Embedding(29 -> 50, float32)
  (B): Embedding(29 -> 50, float32)
  (W): Dense(None -> 29, linear)
  (A): Embedding(29 -> 50, float32)
)


In [19]:
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(model.collect_params(), 'adam', {'learning_rate': 0.01})
log_loss = []
log_perp = []

In [21]:
batch_size = 32
epochs  = 300

In [22]:
epoch_log_loss = []
import math
for epoch in range(epochs):
    dataiter = mx.io.NDArrayIter([inputs_train, queries_train], answers_train, batch_size, shuffle=False, last_batch_handle='discard')
    
    for batch in dataiter:
        log_loss = []
        train_loss = train(model, trainer, 
                           batch.data[0].as_in_context(ctx), 
                           batch.data[1].as_in_context(ctx),
                           batch.label[0].as_in_context(ctx),
                           batch_size)

        # Logging
        log_loss.append([train_loss])#, test_loss])
    
    epoch_log_loss.append([np.mean(log_loss)])
    
    state = { 'epoch': epoch, 'learning_rate': trainer.learning_rate, 'perplexity': math.exp(epoch_log_loss[epoch][0])}
    print(state)
    
    lr_decay = 1.5
    if (len(epoch_log_loss) > 1) and (epoch_log_loss[epoch][0] > epoch_log_loss[epoch-1][0] * 0.9999):
        print ('update learning rate from %.3f to %.3f' % (trainer.learning_rate, trainer.learning_rate/lr_decay))
        trainer.set_learning_rate(trainer.learning_rate / lr_decay)
    if trainer.learning_rate < 1e-5: 
        break

{'epoch': 0, 'perplexity': 6.115564203702698, 'learning_rate': 0.01}
{'epoch': 1, 'perplexity': 5.520157808431111, 'learning_rate': 0.01}
{'epoch': 2, 'perplexity': 5.631711224203392, 'learning_rate': 0.01}
update learning rate from 0.010 to 0.007
{'epoch': 3, 'perplexity': 3.2840659214117527, 'learning_rate': 0.006666666666666667}
{'epoch': 4, 'perplexity': 1.6909363733537708, 'learning_rate': 0.006666666666666667}
{'epoch': 5, 'perplexity': 1.1349282690956812, 'learning_rate': 0.006666666666666667}
{'epoch': 6, 'perplexity': 1.0153575650558406, 'learning_rate': 0.006666666666666667}
{'epoch': 7, 'perplexity': 1.0066820414378812, 'learning_rate': 0.006666666666666667}
{'epoch': 8, 'perplexity': 1.0028816768086037, 'learning_rate': 0.006666666666666667}
{'epoch': 9, 'perplexity': 1.0014552920344273, 'learning_rate': 0.006666666666666667}
{'epoch': 10, 'perplexity': 1.0028680910778285, 'learning_rate': 0.006666666666666667}
update learning rate from 0.007 to 0.004
{'epoch': 11, 'perplex

{'epoch': 94, 'perplexity': 1.0000188573408106, 'learning_rate': 0.0044444444444444444}
{'epoch': 95, 'perplexity': 1.0000183991343015, 'learning_rate': 0.0044444444444444444}
{'epoch': 96, 'perplexity': 1.0000179781797554, 'learning_rate': 0.0044444444444444444}
{'epoch': 97, 'perplexity': 1.000017590751769, 'learning_rate': 0.0044444444444444444}
{'epoch': 98, 'perplexity': 1.0000171586215003, 'learning_rate': 0.0044444444444444444}
{'epoch': 99, 'perplexity': 1.0000168047220073, 'learning_rate': 0.0044444444444444444}
{'epoch': 100, 'perplexity': 1.00001640239307, 'learning_rate': 0.0044444444444444444}
{'epoch': 101, 'perplexity': 1.0000160112403444, 'learning_rate': 0.0044444444444444444}
{'epoch': 102, 'perplexity': 1.000015683416879, 'learning_rate': 0.0044444444444444444}
{'epoch': 103, 'perplexity': 1.0000152885390874, 'learning_rate': 0.0044444444444444444}
{'epoch': 104, 'perplexity': 1.0000149383646926, 'learning_rate': 0.0044444444444444444}
{'epoch': 105, 'perplexity': 1.

{'epoch': 187, 'perplexity': 1.0000029951315188, 'learning_rate': 0.0044444444444444444}
{'epoch': 188, 'perplexity': 1.0000029392519987, 'learning_rate': 0.0044444444444444444}
{'epoch': 189, 'perplexity': 1.000002894548612, 'learning_rate': 0.0044444444444444444}
{'epoch': 190, 'perplexity': 1.0000028461199264, 'learning_rate': 0.0044444444444444444}
{'epoch': 191, 'perplexity': 1.0000027827900406, 'learning_rate': 0.0044444444444444444}
{'epoch': 192, 'perplexity': 1.0000027343613604, 'learning_rate': 0.0044444444444444444}
{'epoch': 193, 'perplexity': 1.000002704558957, 'learning_rate': 0.0044444444444444444}
{'epoch': 194, 'perplexity': 1.0000026449543802, 'learning_rate': 0.0044444444444444444}
{'epoch': 195, 'perplexity': 1.0000025965257067, 'learning_rate': 0.0044444444444444444}
{'epoch': 196, 'perplexity': 1.000002562998235, 'learning_rate': 0.0044444444444444444}
{'epoch': 197, 'perplexity': 1.0000025145695655, 'learning_rate': 0.0044444444444444444}
{'epoch': 198, 'perplexi

{'epoch': 281, 'perplexity': 1.000000596046398, 'learning_rate': 0.0044444444444444444}
{'epoch': 282, 'perplexity': 1.000000588595813, 'learning_rate': 0.0044444444444444444}
{'epoch': 283, 'perplexity': 1.000000581145228, 'learning_rate': 0.0044444444444444444}
{'epoch': 284, 'perplexity': 1.0000005774199356, 'learning_rate': 0.0044444444444444444}
{'epoch': 285, 'perplexity': 1.0000005662440583, 'learning_rate': 0.0044444444444444444}
{'epoch': 286, 'perplexity': 1.0000005513428887, 'learning_rate': 0.0044444444444444444}
{'epoch': 287, 'perplexity': 1.000000543892304, 'learning_rate': 0.0044444444444444444}
{'epoch': 288, 'perplexity': 1.0000005252658426, 'learning_rate': 0.0044444444444444444}
{'epoch': 289, 'perplexity': 1.0000005140900228, 'learning_rate': 0.0044444444444444444}
{'epoch': 290, 'perplexity': 1.0000005066394384, 'learning_rate': 0.0044444444444444444}
{'epoch': 291, 'perplexity': 1.000000495463562, 'learning_rate': 0.0044444444444444444}
{'epoch': 292, 'perplexity

In [33]:
model.collect_params()

memn2n0_ (
  Parameter memn2n0_embedding0_weight (shape=(29, 50), dtype=<class 'numpy.float32'>)
  Parameter memn2n0_embedding1_weight (shape=(29, 50), dtype=<class 'numpy.float32'>)
  Parameter memn2n0_embedding2_weight (shape=(29, 50), dtype=<class 'numpy.float32'>)
  Parameter memn2n0_dense0_weight (shape=(29, 50), dtype=<class 'numpy.float32'>)
  Parameter memn2n0_dense0_bias (shape=(29,), dtype=<class 'numpy.float32'>)
)

In [23]:
pred_ = model(mx.nd.array(inputs_test, mx.cpu()), mx.nd.array(queries_test, mx.cpu()))
pred_ = pred_.asnumpy()

In [24]:
set([np.argmax(i) for i in pred_])

{2, 3, 5, 6, 10, 13}

In [25]:
print (set(np.argmax(i) for i in pred_))
print (set(i for i in answers_test))

{2, 3, 5, 6, 10, 13}
{2, 3, 5, 6, 10, 13}


In [26]:
print ([np.argmax(i) for i in pred_[0:10]])
print ([i for i in answers_test[0:10]])

[6, 2, 10, 6, 10, 6, 5, 6, 13, 13]
[6, 2, 10, 6, 10, 6, 5, 6, 13, 13]


In [27]:
np.sum([bool(np.argmax(i) == answers_test[idx:idx+1]) for idx, i in enumerate(pred_)])/len(pred_)

0.996

In [28]:
## base rate
print (1/len(set(i for i in answers_test)))

0.16666666666666666


In [29]:
idx = 0

In [30]:
for ido in test_data.data_story[idx:idx+1][0]:
    print (" ".join([word_re.get(int(i), '.') for i in ido if int(i) != 0]))
    
print (" ".join([word_re.get(int(i), '.') for i in test_data.data_query[idx:idx+1][0] if int(i) != 0]))
print (word_re.get(int(test_data.data_answer[idx:idx+1][0]), '.'))

john travelled to the hallway .
mary journeyed to the bathroom .








where is john
hallway


In [31]:
print (word_re[ np.argmax(pred_[idx])])

hallway
